In [ ]:
import os
import json
from typing import List
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from bs4 import BeautifulSoup

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
PATH_TO_CHROME_DRIVER = 'B:\\Users\\ekfon\\chromeDriver\\chromedriver.exe'

MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        """
        begin Selenium equivalent of requests:
        response = requests.get(url, headers=headers)
        self.body = response.content #which is then passed on to bs
        """
        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        page_source = driver.page_source
        driver.quit()
        #end Selenium part
        
        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage title: \"{self.title}\"\nWebpage contents:\n{self.text}\n\n"

In [ ]:
#system prompt for the link anthology
anthology_sPrompt = "I'll provide you with a list of links from a webpage. \
You are able to decide which links would be most relevant to include in a brochure about the entity this website is for, \
such as the About page, any personal/company page, or a careers/goals page, if any.\n"

anthology_sPrompt += "You will respond in JSON format, providing full https URLs, just like in this example:\n"

anthology_sPrompt += """
{
    "links": [
        {"type": "about page", "url": "https://www.example-url.com/about"}
        {"type": "careers page", "url": "https://further.example-url.co.uk/Careers/"}
    ]
}
"""

In [ ]:
def get_anthology_user_prompt(website):
    user_prompt = f"Below is the list of links from the webpage {website.url}. "
    user_prompt += "Please decide which of the links are relevant for a brochure presenting what the website is about. \
Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links."
    user_prompt += "Here is the list of links (some might be relative links):\n\n"
    user_prompt += "\n".join(website.links)

    return user_prompt

In [ ]:
def get_links_anthology(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": anthology_sPrompt},
            {"role": "user", "content": get_anthology_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result) #because result is a string, and what we want is an actual dictionary

In [ ]:
def get_all_details(url):
    result = "Landing page:\n\n"
    result += Website(url).get_contents()
    links = get_links_anthology(url)

    for link in links["links"]: #remember that links is a json dictionary
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()

    return result

In [ ]:
contentScan_sPrompt = "You are the world's most powerful content analysis assistant. You are given a selection of the \
most representative pages from a website. You scan and analyze the content to find out if any of the following key principles are violated:\n\n \
- inclusive language (is the language gender-neutral, non-binary, etc.)\n \
- non-violent language (avoid unnecessarily violent language)\n \
- respectful language (avoid stereotypes, othering, be history-conscious)\n\n \
Write a short report in Markdown about the content of the website. Report any violation of the above-mentioned content principles, if any, \
and mention the section where you found it. If possible, help the author of the text re-word or re-phrase the problematic passage. Mention only \
the aspects of the content you would recommend improving. Mention each violation only once. Conclude your analysis with a \
\"language score\" from 30 to 100 based on how much the content principles are respected. Be helpful and non-judgmental."

In [ ]:
def get_contentScan_user_prompt(entity_name, url):
    prompt = f"You are looking at the website of {entity_name}.\n"
    prompt += "Here are the contents of its landing page and other relevant pages. Based on this content, \
create a content report for this website in Markdown, concerning inclusive, non-violent, respectful language, etc.:\n\n"
    prompt += get_all_details(url)
    prompt = prompt[:10_000] #this limits the prompt input, just in case
    
    return prompt

In [ ]:
def create_contentScan(entity_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": contentScan_sPrompt},
        {"role": "user", "content": get_contentCan_user_prompt(entity_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
def stream_contentScan(entity_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": contentScan_sPrompt},
        {"role": "user", "content": get_contentScan_user_prompt(entity_name, url)}
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_contentScan("Acrolinx Website", "https://www.acrolinx.com/")